In [39]:
# DB instance identifier: database-1
# username: admin
# pass: 12345678
# host: database-1.ctmjxsjl9mbd.us-east-1.rds.amazonaws.com
# port: 6033

In [3]:
import csv
import pandas as pd
import string
import io
import os.path
import glob
import pymysql

In [4]:
db = pymysql.connect(user = "admin", 
                host = "database-1.ctmjxsjl9mbd.us-east-1.rds.amazonaws.com",
                password = "12345678",
                port = 6033,
                local_infile=1)

cursor = db.cursor()

In [5]:
# creates/connects entire database and creates MySQL Directory Table and MySQL Partitions Table
# fid|name|parent|parentid|content|file
# pid|fid|tableName|partition_name|part

sql = "DROP DATABASE IF EXISTS mydatabase"
cursor.execute(sql)

sql = "CREATE DATABASE mydatabase"
cursor.execute(sql)

cursor.connection.commit()

sql = "USE mydatabase"
cursor.execute(sql)

sql = "CREATE TABLE dir (fid int AUTO_INCREMENT PRIMARY KEY, name VARCHAR(255), parent VARCHAR(255), parentid int, content VARCHAR(255), file VARCHAR(255))"
cursor.execute(sql)

sql = "CREATE TABLE partitions (pid int AUTO_INCREMENT PRIMARY KEY, file_id int NOT NULL, tableName VARCHAR(255), partition_name VARCHAR(255), part int NOT NULL, FOREIGN KEY (file_id) REFERENCES dir(fid) ON DELETE CASCADE)"
cursor.execute(sql)

sql = "INSERT into dir (name, parent, parentid, content, file) VALUES ('root', '', 0, '/', '')"
cursor.execute(sql)

db.commit()

In [6]:
def dir():
    sql = "select * from dir"
    cursor.execute(sql)
    results = cursor.fetchall()
    df = pd.DataFrame(results, columns=['fid', 'name', 'parent', 'parentid', 'content', 'file'])
    print(df)
    db.commit()
dir()

   fid  name parent  parentid content file
0    1  root                0       /     


In [13]:
def partitions():
    sql = "select * from partitions"
    cursor.execute(sql)
    results = cursor.fetchall()
    df = pd.DataFrame(results, columns=['pid', 'file_id', 'tableName', 'partition_name', 'part'])
    print(df)
    db.commit()
partitions()

Empty DataFrame
Columns: [pid, file_id, tableName, partition_name, part]
Index: []


In [14]:
def fileExists(filename):
    if("/" not in filename):
        filename = "/" + filename
    cursor.execute("SELECT * FROM dir WHERE content LIKE '" + filename + "'")
    f = cursor.fetchall()
    if(len(f) != 0):
        return 1
    return 0

def partitionExists(table, partitions):
    cursor.execute("SELECT * FROM partitions WHERE tableName LIKE '" + table +"' AND part = " + str(partitions))
    result = cursor.fetchall()
    if(len(result) == 0):
        return 0
    return 1

In [15]:
def mkdir(dirname):
    if(fileExists(dirname)):
        return "file already exists"
    if("/" not in dirname):
        name = dirname
        parent = "root"
        parentid = '1'
        content = "/" + dirname
    else:
        name = dirname.split('/')[-1]
        content = dirname
        if(dirname.split('/')[-2]):
            parent = dirname.split('/')[-2]
            if(not fileExists(parent)):
                return "parent directory not made"
            cursor.execute("SELECT fid FROM dir WHERE name LIKE '" + parent + "'")
            result = cursor.fetchall()
            parentid = result[0][0]
        else:
            parent = "root"
            parentid = '1'
    file = ""
    sql = "INSERT INTO dir (name, parent, parentid, content, file) VALUES (%s, %s, %s, %s, %s)"
    val = (name, parent, int(parentid), content, file)
    cursor.execute(sql, val)
    db.commit()

In [16]:
mkdir("/user")
mkdir("/user/raaj")

'file already exists'

In [17]:
dir()

   fid  name parent  parentid     content file
0    1  root                0           /     
1    2  user   root         1       /user     
2    3  raaj   user         2  /user/raaj     


In [18]:
def ls(dirname):
    cursor.execute("SELECT fid FROM dir WHERE content LIKE '"+ dirname +"'")
    result = cursor.fetchall()
    dirid = result[0][0]
    sql = "SELECT name FROM dir WHERE parentid = " + str(dirid) 
    cursor.execute(sql)
    myresult = cursor.fetchall()
    db.commit()
    print("files in directory '" + dirname + "':\n")
    for x in myresult:
      print(x[0])

In [19]:
ls('/')

files in directory '/':

user


In [20]:
def rm(filename):
    if("/" not in filename):
        filename = "/" + filename
    sql = "DELETE FROM dir WHERE content LIKE '" + filename + "'";
    cursor.execute(sql)
    db.commit()

In [21]:
rm("/user/raaj")
ls("/user")

files in directory '/user':



In [22]:
def put(filename, dirname, k):
    # read file
    data = pd.read_csv(filename) 
    data.to_csv("sqltable.csv")
    df = pd.DataFrame(data)
    tableName = filename.split('/')[-1]
    tableName = tableName.split('.')[-2]
    cursor.execute("DROP TABLE IF EXISTS " + tableName)
    db.commit()

    # create file in dir and partition table
    if(dirname == "/" or dirname == ""):
        name = filename.split('/')[-1]
        content = "/" + filename.split('/')[-1]
        parent = "root"
        parentid = "1"
    elif("/" not in dirname):
        name = filename.split('/')[-1]
        content = "/" + dirname + "/" + filename.split('/')[-1]
        parent = dirname
        cursor.execute("SELECT fid FROM dir WHERE name LIKE '" + parent + "'")
        result = cursor.fetchall()
        parentid = result[0][0]
    else:
        name = filename.split('/')[-1]
        content = dirname + "/" + filename.split('/')[-1]
        if(dirname.split('/')[-1]):
            parent = dirname.split('/')[-1]
            cursor.execute("SELECT fid FROM dir WHERE name LIKE '" + parent + "'")
            result = cursor.fetchall()
            parentid = result[0][0]
        else:
            parent = "root"
            parentid = "1"

    file = tableName
    if(fileExists(content)):
        return "file already exists there"
    sql = "INSERT INTO dir (name, parent, parentid, content, file) VALUES (%s, %s, %s, %s, %s)"
    val = (name, parent, int(parentid), content, file)
    cursor.execute(sql, val)
    db.commit()
    
    
    # create table
    print("processing " + tableName + " into directory...")
    sql = "CREATE TABLE " + tableName + " (tid int AUTO_INCREMENT PRIMARY KEY) PARTITION BY KEY(tid) PARTITIONS " + str(k)
    cursor.execute(sql)
    db.commit()
    prev_col = "tid"
    columns = ""
    questionMarks = ""
    for col in df.columns:
        col = col.translate(str.maketrans('', '', string.punctuation)).replace(" ", "")
        if(col == "Long"):
            col = "longi"
        sql = "ALTER TABLE " + tableName + " ADD COLUMN " + col + " VARCHAR(255) AFTER " + prev_col
        cursor.execute(sql)
        db.commit()
        prev_col = col
        columns = columns + col + ","
        questionMarks = questionMarks + "%s,"
    file = open(filename)
    csv_data = csv.reader(file)
    skipHeader = True
    
    
    sql = "LOAD DATA LOCAL INFILE 'sqltable.csv' INTO TABLE " + tableName +  " FIELDS TERMINATED BY ',' ENCLOSED BY '\"' LINES TERMINATED BY '\n' IGNORE 1 ROWS"
    cursor.execute(sql)

    '''
    # insert data into table 
    for row in csv_data:
        if skipHeader:
            skipHeader = False
            continue
        cursor.execute('INSERT INTO ' + tableName + ' ('+  columns[:-1] + ')' 'VALUES (' + questionMarks[:-1] + ')', row)
        db.commit()
    '''
    # create partition tables
    sql = "SELECT fid FROM dir WHERE content LIKE '" + content + "'"
    cursor.execute(sql)
    
    fid = cursor.fetchall()
    
    for x in range(k):
        partitionName = tableName + "_" + str(x)
        cursor.execute("DROP TABLE IF EXISTS " + partitionName)
        cursor.execute("CREATE TABLE " + partitionName + " SELECT * FROM " + tableName + " PARTITION(p" + str(x) + ")")
        sql = "INSERT INTO partitions (file_id, tableName, partition_name, part) VALUES (" + str(fid[-1][0]) + ", '" + tableName + "','" + partitionName + "', " + str(x+1) + ")"
        cursor.execute(sql)
        db.commit()
    print("done")
dir()

   fid  name parent  parentid content file
0    1  root                0       /     
1    2  user   root         1   /user     


In [23]:
# load data sets

mkdir('/covid')
#put("covid/country_wise_latest.csv", "/covid", 2)
#put("covid/full_grouped.csv", "/covid", 2)
#put("covid/usa_county_wise.csv", "/covid", 2)
#put("covid/worldometer_data.csv", "/covid", 3)
put("covid/covid_19_clean_complete.csv", "/covid", 1)
#put("covid/day_wise.csv", "/covid", 1)

mkdir('/ebola')
put("ebola/ebola_2014_2016_clean.csv", "/ebola", 4)

mkdir('/sars')
put("sars/sars_2003_complete_dataset_clean.csv", "/sars", 5)

dir()

processing covid_19_clean_complete into directory...
done
processing ebola_2014_2016_clean into directory...
done
processing sars_2003_complete_dataset_clean into directory...
done
   fid                                  name parent  parentid  \
0    1                                  root                0   
1    2                                  user   root         1   
2    4                                 covid   root         1   
3    5           covid_19_clean_complete.csv  covid         4   
4    6                                 ebola   root         1   
5    7             ebola_2014_2016_clean.csv  ebola         6   
6    8                                  sars   root         1   
7    9  sars_2003_complete_dataset_clean.csv   sars         8   

                                      content  \
0                                           /   
1                                       /user   
2                                      /covid   
3          /covid/covid_19_clean_comp

In [24]:
def cat(filename):
    # check if file exists
    if(not fileExists(filename)):
        return "file does not exist"
    
    # get table name and columns 
    sql = "SELECT file FROM dir WHERE content LIKE '" + filename + "'"
    cursor.execute(sql)
    tableName = cursor.fetchall()
    cursor.execute("SHOW columns FROM "+ tableName[0][0])
    columns = [column[0] for column in cursor.fetchall()]
    
    # display table
    sql = "SELECT * FROM " + tableName[0][0]
    cursor.execute(sql)
    results = cursor.fetchall()
    df = pd.DataFrame(results, columns=columns)
    print(df)
    db.commit()

In [25]:
cat("/sars/sars_2003_complete_dataset_clean.csv")

       tid        Date               Country Cumulativenumberofcases  \
0      123  2003-03-27               Romania                       3   
1      127  2003-03-27        United Kingdom                       3   
2      131  2003-03-28                 China                     806   
3      135  2003-03-28               Germany                       4   
4      139  2003-03-28             Singapore                      86   
...    ...         ...                   ...                     ...   
2532  2509  2003-07-11                 China                    5327   
2533  2510  2003-07-11  Hong Kong SAR, China                    1755   
2534  2511  2003-07-11      Macao SAR, China                       1   
2535  2512  2003-07-11         Taiwan, China                     671   
2536  2520  2003-07-11                Kuwait                       1   

     Numberofdeaths Numberrecovered  
0                 0               0  
1                 0               0  
2                34  

In [33]:
def getPartitionLocations(filename):
    # check for file validity
    if(not fileExists(filename)):
        return "file does not exist"
    if("/" not in filename):
        filename = "/" + filename
        
    # get file id
    sql = "SELECT fid FROM dir WHERE content LIKE '" + filename + "'"
    cursor.execute(sql)
    fid = cursor.fetchall()
    
    # get partitions
    sql = "SELECT pid FROM partitions WHERE file_id LIKE " + str(fid[-1][0]) 
    cursor.execute(sql)
    
    result = cursor.fetchall()
    return result

In [34]:
def readPartition(filename, partition):
    # check validity of file
    if(not fileExists(filename)):
        return "file does not exist"
    if("/" not in filename):
        filename = "/" + filename
    
    # find file id
    sql = "SELECT file FROM dir WHERE content LIKE '" + filename + "'"
    cursor.execute(sql)
    tableName = cursor.fetchall()
    
    # get partition name
    if(not partitionExists(str(tableName[0][0]), partition)):
        return "partition does not exist"
    sql = "SELECT partition_name FROM partitions WHERE tableName LIKE '" + str(tableName[0][0]) +"' AND part = " + str(partition)
    cursor.execute(sql)
    db.commit()
    partitionTable = cursor.fetchall()
    cursor.execute("SHOW columns FROM "+ partitionTable[0][0])
    columns = [column[0] for column in cursor.fetchall()]
    
    sql = "SELECT * FROM " + str(partitionTable[0][0])
    cursor.execute(sql)
    result = cursor.fetchall()
    df = pd.DataFrame(result, columns=columns)
    print(df)
    db.commit()

In [38]:
def search_deathsByCountryAndDataset(country, disease):
    total = 0
    
    if(disease == "ebola"):
        # ebola_2014_2016_clean.csv DEATHS ARE CUMALITIVE?
        print("\nGetting partition ids for 'ebola_2014_2016_clean.csv'...\n")
        x = len(getPartitionLocations('/ebola/ebola_2014_2016_clean.csv'))

        # find file id
        filename = '/ebola/ebola_2014_2016_clean.csv'
        sql = "SELECT file FROM dir WHERE content LIKE '" + filename + "'"
        cursor.execute(sql)
        tableName = cursor.fetchall()
        dfs = []
        for i in range(x):
            # get partition name
            if(not partitionExists(str(tableName[0][0]), i+1)):
                return "partition does not exist"
            sql = "SELECT partition_name FROM partitions WHERE tableName LIKE '" + str(tableName[0][0]) +"' AND part = " + str(i+1)
            cursor.execute(sql)
            db.commit()
            partitionTable = cursor.fetchall()

            sql = "SELECT Country, Cumulativenoofconfirmedprobableandsuspecteddeaths FROM " + str(partitionTable[0][0]) + " WHERE Country LIKE '" + country + "' GROUP BY Country"
            cursor.execute(sql)
            result = cursor.fetchall()
            df = pd.DataFrame(result)
            if(not df.empty):
                df.columns = ["country", "deaths"]
                df['deaths'] = df['deaths'].astype('float')
            dfs.append(df)
            db.commit()
        fdf = pd.concat(dfs)
        print(fdf, "\n")
        if(not fdf.empty):
            r = fdf.groupby(["country"])['deaths'].max().reset_index()
            print(r)
            total = total + r['deaths'][0]
    elif(disease == "sars"):
        # sars/sars_2003_complete_dataset_clean.csv 
        print("\nGetting partition ids for 'sars_2003_complete_dataset_clean.csv'...\n")
        x = len(getPartitionLocations('/sars/sars_2003_complete_dataset_clean.csv'))

        # find file id
        filename = '/sars/sars_2003_complete_dataset_clean.csv'
        sql = "SELECT file FROM dir WHERE content LIKE '" + filename + "'"
        cursor.execute(sql)
        tableName = cursor.fetchall()
        dfs = []
        for i in range(x):
            # get partition name
            if(not partitionExists(str(tableName[0][0]), i+1)):
                return "partition does not exist"
            sql = "SELECT partition_name FROM partitions WHERE tableName LIKE '" + str(tableName[0][0]) +"' AND part = " + str(i+1)
            cursor.execute(sql)
            db.commit()
            partitionTable = cursor.fetchall()

            sql = "SELECT Country, Date, Numberofdeaths FROM " + str(partitionTable[0][0]) + " WHERE Country LIKE '" + country + "'  GROUP BY Country"
            cursor.execute(sql)
            result = cursor.fetchall()
            df = pd.DataFrame(result)
            if(not df.empty):
                df.columns = ["country", "deaths"]
                df['deaths'] = df['deaths'].astype('int')
            dfs.append(df)
            db.commit()
        fdf = pd.concat(dfs)
        print(fdf, "\n")
        if(not fdf.empty):
            r = fdf.groupby(["country"])['deaths'].sum().reset_index()
            print(r)
            total = total + r['deaths'][0]
    else:
         # covid19_clean_complete
        print("\nGetting partition ids for 'covid19_clean_complete.csv'...\n")
        x = len(getPartitionLocations('/covid/covid_19_clean_complete.csv'))

        # find file id
        filename = '/covid/covid_19_clean_complete.csv'
        sql = "SELECT file FROM dir WHERE content LIKE '" + filename + "'"
        cursor.execute(sql)
        tableName = cursor.fetchall()
        dfs = []
        for i in range(x):
            # get partition name
            if(not partitionExists(str(tableName[0][0]), i+1)):
                return "partition does not exist"
            sql = "SELECT partition_name FROM partitions WHERE tableName LIKE '" + str(tableName[0][0]) +"' AND part = " + str(i+1)
            cursor.execute(sql)
            db.commit()
            partitionTable = cursor.fetchall()

            sql = "SELECT CountryRegion, Deaths FROM " + str(partitionTable[0][0]) + " WHERE CountryRegion LIKE '" + country + "'  GROUP BY CountryRegion"
            cursor.execute(sql)
            result = cursor.fetchall()
            df = pd.DataFrame(result)
            if(not df.empty):
                df.columns = ["country", "deaths"]
                df['deaths'] = df['deaths'].astype('int')
            dfs.append(df)
            db.commit()
        fdf = pd.concat(dfs)
        print(fdf,"\n")
        if(not fdf.empty):
            r = fdf.groupby(["country"])['deaths'].sum().reset_index()
            print(r)
            total = total + r['deaths'][0]
        
    
    print("\n\nTotal Deaths for " + country + ": " + str(total))
    

In [41]:
search_deathsByCountryAndDataset("Nigeria", "covid")

# search_deathsByCountryAndDataset("India", "ebola")

# search_deathsByCountryAndDataset("India", "sars")



Getting partition ids for 'covid19_clean_complete.csv'...

   country  deaths
0  Nigeria       0 

   country  deaths
0  Nigeria       0


Total Deaths for Nigeria: 0


In [68]:
def countries_casesBetweenXY():
    # ebola_2014_2016_clean.csv DEATHS ARE CUMALITIVE?
    print("\nGetting partition ids for 'ebola_2014_2016_clean.csv'...\n")
    x = len(getPartitionLocations('/ebola/ebola_2014_2016_clean.csv'))
    total = []
    # find file id
    filename = '/ebola/ebola_2014_2016_clean.csv'
    sql = "SELECT file FROM dir WHERE content LIKE '" + filename + "'"
    cursor.execute(sql)
    tableName = cursor.fetchall()
    dfs = []
    for i in range(x):
        # get partition name
        if(not partitionExists(str(tableName[0][0]), i+1)):
            return "partition does not exist"
        sql = "SELECT partition_name FROM partitions WHERE tableName LIKE '" + str(tableName[0][0]) +"' AND part = " + str(i+1)
        cursor.execute(sql)
        db.commit()
        partitionTable = cursor.fetchall()

        sql = "SELECT Country, SUM(Cumulativenoofconfirmedprobableandsuspectedcases) FROM " + str(partitionTable[0][0]) + " GROUP BY Country"
        cursor.execute(sql)
        result = cursor.fetchall()
        df = pd.DataFrame(result)
        if(not df.empty):
            df.columns = ["country", "cases"]
        dfs.append(df)
        db.commit()
    fdf = pd.concat(dfs)
    if(not fdf.empty):
        r = fdf.groupby(["country"])['cases'].sum().reset_index()
    print(fdf)
    total.append(r)
    
    # sars/sars_2003_complete_dataset_clean.csv
    print("\nGetting partition ids for 'sars/sars_2003_complete_dataset_clean.csv'...\n")
    x = len(getPartitionLocations('/sars/sars_2003_complete_dataset_clean.csv'))
    
    # find file id
    filename = '/sars/sars_2003_complete_dataset_clean.csv'
    sql = "SELECT file FROM dir WHERE content LIKE '" + filename + "'"
    cursor.execute(sql)
    tableName = cursor.fetchall()
    dfs = []
    for i in range(x):
        # get partition name
        if(not partitionExists(str(tableName[0][0]), i+1)):
            return "partition does not exist"
        sql = "SELECT partition_name FROM partitions WHERE tableName LIKE '" + str(tableName[0][0]) +"' AND part = " + str(i+1)
        cursor.execute(sql)
        db.commit()
        partitionTable = cursor.fetchall()

        sql = "SELECT Country, SUM(Cumulativenumberofcases) FROM " + str(partitionTable[0][0]) + " GROUP BY Country"
        cursor.execute(sql)
        result = cursor.fetchall()
        df = pd.DataFrame(result)
        if(not df.empty):
            df.columns = ["country", "cases"]
        dfs.append(df)
        db.commit()
    fdf = pd.concat(dfs)
    if(not fdf.empty):
        r = fdf.groupby(["country"])['cases'].sum().reset_index()
    print(fdf)
    total.append(r)
    
    
    # covid/covid_19_clean_complete.csv
    print("\nGetting partition ids for 'covid/covid_19_clean_complete.csv'...\n")
    x = len(getPartitionLocations('/covid/covid_19_clean_complete.csv'))
    
    # find file id
    filename = '/covid/covid_19_clean_complete.csv'
    sql = "SELECT file FROM dir WHERE content LIKE '" + filename + "'"
    cursor.execute(sql)
    tableName = cursor.fetchall()
    dfs = []
    for i in range(x):
        # get partition name
        if(not partitionExists(str(tableName[0][0]), i+1)):
            return "partition does not exist"
        sql = "SELECT partition_name FROM partitions WHERE tableName LIKE '" + str(tableName[0][0]) +"' AND part = " + str(i+1)
        cursor.execute(sql)
        db.commit()
        partitionTable = cursor.fetchall()

        sql = "SELECT CountryRegion, SUM(Confirmed) FROM " + str(partitionTable[0][0]) + " GROUP BY CountryRegion"
        cursor.execute(sql)
        result = cursor.fetchall()
        df = pd.DataFrame(result)
        if(not df.empty):
            df.columns = ["country", "cases"]
        dfs.append(df)
        db.commit()
    fdf = pd.concat(dfs)
    if(not fdf.empty):
        r = fdf.groupby(["country"])['cases'].sum().reset_index()
    print(fdf)
    total.append(r)
    
    print("\n\n Countries with cases between 2000 and 5000")
    final = pd.concat(total)
    final.columns = ["country", "cases"]
    filtering = final.groupby(["country"])['cases'].sum().reset_index()
    print(filtering.query('cases > 2000 & cases < 5000'))
    

In [67]:
countries_casesBetweenXY()


Getting partition ids for 'ebola_2014_2016_clean.csv'...

partition ID(s): 
2
3
4
5


                    country      cases
0                    Guinea   144462.0
1                   Nigeria      623.0
2                   Senegal       49.0
3                   Liberia   343086.0
4  United States of America      134.0
5              Sierra Leone   447325.0
6                     Spain       34.0
7                      Mali      342.0
8            United Kingdom       36.0
9                     Italy       19.0
0              Sierra Leone  1361776.0
1                   Senegal      104.0
2                   Liberia  1127518.0
3                   Nigeria     2422.0
4                    Guinea   351543.0
5  United States of America      470.0
6                     Spain      104.0
7                      Mali      808.0
8            United Kingdom      102.0
9                     Italy       64.0
0                   Liberia   391169.0
1                    Guinea   143480.0
2              S

In [60]:
def totalNumberofDeathsPerCountry():
    # ebola_2014_2016_clean.csv DEATHS ARE CUMALITIVE?
    print("\nGetting partition ids for 'ebola_2014_2016_clean.csv'...\n")
    x = len(getPartitionLocations('/ebola/ebola_2014_2016_clean.csv'))
    total = []
    # find file id
    filename = '/ebola/ebola_2014_2016_clean.csv'
    sql = "SELECT file FROM dir WHERE content LIKE '" + filename + "'"
    cursor.execute(sql)
    tableName = cursor.fetchall()
    dfs = []
    for i in range(x):
        # get partition name
        if(not partitionExists(str(tableName[0][0]), i+1)):
            return "partition does not exist"
        sql = "SELECT partition_name FROM partitions WHERE tableName LIKE '" + str(tableName[0][0]) +"' AND part = " + str(i+1)
        cursor.execute(sql)
        db.commit()
        partitionTable = cursor.fetchall()

        sql = "SELECT Country, SUM(Cumulativenoofconfirmedprobableandsuspecteddeaths) FROM " + str(partitionTable[0][0]) + " GROUP BY Country"
        cursor.execute(sql)
        result = cursor.fetchall()
        df = pd.DataFrame(result)
        if(not df.empty):
            df.columns = ["country", "deaths"]
        dfs.append(df)
        db.commit()
    fdf = pd.concat(dfs)
    if(not fdf.empty):
        r = fdf.groupby(["country"])['deaths'].sum().reset_index()
        print(r)
    total.append(r)
    
    # sars/sars_2003_complete_dataset_clean.csv
    print("\nGetting partition ids for 'sars/sars_2003_complete_dataset_clean.csv'...\n")
    x = len(getPartitionLocations('/sars/sars_2003_complete_dataset_clean.csv'))
    
    # find file id
    filename = '/sars/sars_2003_complete_dataset_clean.csv'
    sql = "SELECT file FROM dir WHERE content LIKE '" + filename + "'"
    cursor.execute(sql)
    tableName = cursor.fetchall()
    dfs = []
    for i in range(x):
        # get partition name
        if(not partitionExists(str(tableName[0][0]), i+1)):
            return "partition does not exist"
        sql = "SELECT partition_name FROM partitions WHERE tableName LIKE '" + str(tableName[0][0]) +"' AND part = " + str(i+1)
        cursor.execute(sql)
        db.commit()
        partitionTable = cursor.fetchall()

        sql = "SELECT Country, SUM(Numberofdeaths) FROM " + str(partitionTable[0][0]) + " GROUP BY Country"
        cursor.execute(sql)
        result = cursor.fetchall()
        df = pd.DataFrame(result)
        if(not df.empty):
            df.columns = ["country", "deaths"]
        dfs.append(df)
        db.commit()
    fdf = pd.concat(dfs)
    if(not fdf.empty):
        r = fdf.groupby(["country"])['deaths'].sum().reset_index()
        print(r)
    total.append(r)
    
    # covid/covid_19_clean_complete.csv
    print("\nGetting partition ids for 'covid/covid_19_clean_complete.csv'...\n")
    x = len(getPartitionLocations('/covid/covid_19_clean_complete.csv'))
    
    # find file id
    filename = '/covid/covid_19_clean_complete.csv'
    sql = "SELECT file FROM dir WHERE content LIKE '" + filename + "'"
    cursor.execute(sql)
    tableName = cursor.fetchall()
    dfs = []
    for i in range(x):
        # get partition name
        if(not partitionExists(str(tableName[0][0]), i+1)):
            return "partition does not exist"
        sql = "SELECT partition_name FROM partitions WHERE tableName LIKE '" + str(tableName[0][0]) +"' AND part = " + str(i+1)
        cursor.execute(sql)
        db.commit()
        partitionTable = cursor.fetchall()

        sql = "SELECT CountryRegion, SUM(Deaths) FROM " + str(partitionTable[0][0]) + " GROUP BY CountryRegion"
        cursor.execute(sql)
        result = cursor.fetchall()
        df = pd.DataFrame(result)
        if(not df.empty):
            df.columns = ["country", "deaths"]
        dfs.append(df)
        db.commit()
    fdf = pd.concat(dfs)
    if(not fdf.empty):
        r = fdf.groupby(["country"])['deaths'].sum().reset_index()
    print(fdf)
    total.append(r)
    
    final = pd.concat(total)
    final.columns = ["country", "deaths"]
    filtering = final.groupby(["country"])['deaths'].sum().reset_index()
    print("\n\nTotal number of deaths per country...")
    print(filtering)
    
        

In [61]:
totalNumberofDeathsPerCountry()


Getting partition ids for 'ebola_2014_2016_clean.csv'...

partition ID(s): 
2
3
4
5


                    country     deaths
0                    Guinea   563069.0
1                     Italy        0.0
2                   Liberia  1099833.0
3                      Mali     1427.0
4                   Nigeria     2032.0
5                   Senegal        0.0
6              Sierra Leone   888834.0
7                     Spain        0.0
8            United Kingdom        0.0
9  United States of America      242.0

Getting partition ids for 'sars/sars_2003_complete_dataset_clean.csv'...

partition ID(s): 
6
7
8
9
10


                 country   deaths
0              Australia      0.0
1                Belgium      0.0
2                 Brazil      0.0
3               Bulgaria      0.0
4                 Canada   2013.0
5                  China  19738.0
6               Colombia      0.0
7                Finland      0.0
8                 France      1.0
9                Germany      0.0
10  

In [129]:
def averageNumberofRecoveredPerOutbreak():
    
    x = len(getPartitionLocations('/ebola/ebola_2014_2016_clean.csv'))
    # find file id
    filename = '/ebola/ebola_2014_2016_clean.csv'
    sql = "SELECT file FROM dir WHERE content LIKE '" + filename + "'"
    cursor.execute(sql)
    tableName = cursor.fetchall()
    dfs = []
    parts = []
    ebola_final = [0,0]
    for i in range(x):
        # get partition name
        if(not partitionExists(str(tableName[0][0]), i+1)):
            return "partition does not exist"
        sql = "SELECT partition_name FROM partitions WHERE tableName LIKE '" + str(tableName[0][0]) +"' AND part = " + str(i+1)
        cursor.execute(sql)
        db.commit()
        partitionTable = cursor.fetchall()
        cursor.execute("ALTER TABLE "+ str(partitionTable[0][0]) + " MODIFY Cumulativenoofconfirmedprobableandsuspectedcases INTEGER")
        sql = "SELECT Country, MAX(Cumulativenoofconfirmedprobableandsuspectedcases), Cumulativenoofconfirmedprobableandsuspectedcases - Cumulativenoofconfirmedprobableandsuspecteddeaths as recovered FROM " + str(partitionTable[0][0]) + " GROUP BY Country"
        cursor.execute(sql)
        result = cursor.fetchall()
        df = pd.DataFrame(result)
        if(not df.empty):
            df.columns = ["country", "cases", "recovered"]
            part = [int(df['recovered'].mean()), int(df['cases'].mean())]
            ebola_final[0] = ebola_final[0] + df['recovered'].mean()
            ebola_final[1] = ebola_final[1] + df['cases'].mean()
            parts.append(part)
        dfs.append(df)
        db.commit()
    ebola_parts = parts
    ebola_final[0] = int(ebola_final[0]/x)
    ebola_final[1] = int(ebola_final[1]/x)

    # sars/sars_2003_complete_dataset_clean.csv
    x = len(getPartitionLocations('/sars/sars_2003_complete_dataset_clean.csv'))
    parts = []
    sars_final = [0,0]
    # find file id
    filename = '/sars/sars_2003_complete_dataset_clean.csv'
    sql = "SELECT file FROM dir WHERE content LIKE '" + filename + "'"
    cursor.execute(sql)
    tableName = cursor.fetchall()
    dfs = []
    for i in range(x):
        # get partition name
        if(not partitionExists(str(tableName[0][0]), i+1)):
            return "partition does not exist"
        sql = "SELECT partition_name FROM partitions WHERE tableName LIKE '" + str(tableName[0][0]) +"' AND part = " + str(i+1)
        cursor.execute(sql)
        db.commit()
        partitionTable = cursor.fetchall()

        sql = "SELECT AVG(Cumulativenumberofcases), AVG(Numberrecovered) FROM " + str(partitionTable[0][0]) 
        cursor.execute(sql)
        result = cursor.fetchall()
        df = pd.DataFrame(result)
        if(not df.empty):
            df.columns = ["cases", "recovered"]
            part = [int(df['recovered'].mean()), int(df['cases'].mean())]
            sars_final[0] = sars_final[0] + df['recovered'].mean()
            sars_final[1] = sars_final[1] + df['cases'].mean()
            parts.append(part)
        dfs.append(df)
        db.commit()
    sars_parts = parts
    sars_final[0] = int(sars_final[0]/x)
    sars_final[1] = int(sars_final[1]/x)
    
    # covid/covid_19_clean_complete.csv
    x = len(getPartitionLocations('/covid/covid_19_clean_complete.csv'))
    parts = []
    covid_final = [0,0]
    # find file id
    filename = '/covid/covid_19_clean_complete.csv'
    sql = "SELECT file FROM dir WHERE content LIKE '" + filename + "'"
    cursor.execute(sql)
    tableName = cursor.fetchall()
    dfs = []
    for i in range(x):
        # get partition name
        if(not partitionExists(str(tableName[0][0]), i+1)):
            return "partition does not exist"
        sql = "SELECT partition_name FROM partitions WHERE tableName LIKE '" + str(tableName[0][0]) +"' AND part = " + str(i+1)
        cursor.execute(sql)
        db.commit()
        partitionTable = cursor.fetchall()

        sql = "SELECT AVG(Confirmed), AVG(Recovered) FROM " + str(partitionTable[0][0])
        cursor.execute(sql)
        result = cursor.fetchall()
        df = pd.DataFrame(result)
        if(not df.empty):
            df.columns = ["cases", "recovered"]
            part = [int(df['recovered'].mean()), int(df['cases'].mean())]
            covid_final[0] = covid_final[0] + df['recovered'].mean()
            covid_final[1] = covid_final[1] + df['cases'].mean()
            parts.append(part)
        dfs.append(df)
        db.commit()
    covid_parts = parts
    covid_final[0] = int(covid_final[0]/x)
    covid_final[1] = int(covid_final[1]/x)
    
    final = {"mapper": {"sars": sars_parts, "ebola": ebola_parts, "covid": covid_parts},"reducer":{"sars": sars_final, "ebola": ebola_final, "covid": covid_final}}
    print(final)

In [130]:
averageNumberofRecoveredPerOutbreak()

{'mapper': {'sars': [[175, 275], [107, 154], [142, 264], [100, 177], [154, 238]], 'ebola': [[299, 2863], [205, 2863], [184, 2861], [171, 2863]], 'covid': [[7915, 16885]]}, 'reducer': {'sars': [136, 222], 'ebola': [215, 2863], 'covid': [7915, 16885]}}
